In [154]:
import math
import numpy as np 
#import matplotlib.colors as cs 
from PIL import Image, ImageColor
from scipy.io import wavfile
import matplotlib.pyplot as plt
import PIL

In [93]:
a, b = wavfile.read('test.wav')

In [96]:
len(b)

31151

In [99]:
res = math.ceil(np.sqrt(len(b)))
delta = res**2 - len(b)
delta

178

In [120]:
type(wavfile.write)

function

In [147]:
def encode(file: str) -> PIL.PngImagePlugin.PngImageFile:
    _ , s_arr = wavfile.read(file)
    resolution = math.ceil(np.sqrt(len(s_arr)))
    delta_res = resolution**2 - len(s_arr)
    
    new_arr = []

    buffer_symbols = ['a', 'b', 'c', 'd', 'e']
    
    for elem in s_arr:
        
        app = None
        num = 0
        
        if elem >= 0:
            
            app = f'#{elem}' + ''.join([np.random.choice(['a', 'b', 'c', 'd', 'e', 1, 2, 3, 4, 5, 6, 7, 8]) for x in range(6 - len(str(elem)))])
            new_arr.append(app)
            num += 1  
        else:
            
            app = f'#f{elem*-1}' + ''.join([np.random.choice(buffer_symbols) for _ in range(6-len(str(elem)))])
            new_arr.append(app)
            num += 1
            
        #f'{np.random.choice(buffer_symbols)}'*(6 - len(str(elem)))
    p_arr = np.array([list(ImageColor.getcolor(x, 'RGB')) for x in new_arr] + [[0,0,0] for x in range(delta_res)])
    
    print(p_arr.shape)
    print(new_arr)
    
    p_arr = p_arr.reshape(resolution, resolution, 3)
    p_arr = p_arr.astype(np.uint8)
    
    img = Image.fromarray(p_arr)
    img.save(f'{file[:-4]}_enco.png')

In [ ]:
encode('test.wav')

In [137]:
imh = np.array(Image.open('test_enco.png'))
imh = imh.reshape(imh.shape[0]**2, 3)

In [151]:
imh[0]

array([ 8, 37, 75], dtype=uint8)

In [152]:
hex_ = 

ValueError: RGBA values should be within 0-1 range

In [ ]:
def decode(path: str):
    